# Import Libraries

In [1]:
from ibm_watson import ToneAnalyzerV3
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson import ApiException
import json
import os
import pandas as pd
from langdetect import detect
import random
import re
import time
from sklearn.model_selection import train_test_split
import ast

# Reading the file

In [2]:
#lockdown1 Duration- 20th March to 14th April
df=pd.read_csv("covid1.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8528 entries, 0 to 8527
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   User       8528 non-null   object
 1   Text       8528 non-null   object
 2   Date       8528 non-null   object
 3   Retweets   8528 non-null   int64 
 4   Favorites  8528 non-null   int64 
 5   HashTags   8521 non-null   object
 6   City       8528 non-null   object
dtypes: int64(2), object(5)
memory usage: 299.9+ KB


,User,Text,Date,Retweets,Favorites,HashTags,City
0,bimaldas1965,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai
1,freddzo,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai
2,Andy_gawai,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai
3,AkshayMahapadi,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai
4,him_paliwal,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai


# Cleaning

In [3]:
def clean_tweet(tweet):
    return ' '.join(re.sub('(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)', ' ', tweet).split())

In [4]:
df['clean_tweet'] = df['Text'].apply(lambda x: clean_tweet(x))
df.head()
df.to_csv('covidclean1.csv',index=False)

In [5]:
data = pd.read_csv('covidclean1.csv')
data= data.iloc[0:500,]
data


,User,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet
0,bimaldas1965,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai,Pandemic is the result of loss of natural boun...
1,freddzo,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai,they will ask me to visit the nearest branch a...
2,Andy_gawai,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai,No schedule for sleeping getting up having foo...
3,AkshayMahapadi,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai,ji please declare d amendnent in ARP over to 3...
4,him_paliwal,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai,Have we got vaccine for Covid19 If not then wh...
...,...,...,...,...,...,...,...,...
495,COVID19UPDATE18,6 more #COVID19 positive cases ( 5 in Nainital...,2020-04-04 12:22:27+00:00,0,0,#COVID19,Mumbai,6 more COVID19 positive cases 5 in Nainital am...
496,mayankbhagwat,#COVID2019 #COVID19Pandemic #Mumbai 48-year-Ol...,2020-04-04 12:15:34+00:00,0,2,#COVID2019 #COVID19Pandemic #Mumbai #Mukund #D...,Mumbai,COVID2019 COVID19Pandemic Mumbai 48 year Old M...
497,Dr_IshanThakkar,Currently we are seeing 2 epidemics... One of ...,2020-04-04 11:31:09+00:00,0,1,#COVID19 #STUPIDITY20,Mumbai,Currently we are seeing 2 epidemics One of cor...
498,diptensworld,"Stay home, stay safe! Brahminy Starling #color...",2020-04-04 11:12:14+00:00,1,2,#colors #beauty #nature #birds #birdwatching #...,Mumbai,Stay home stay safe Brahminy Starling colors a...


# IBM Tone Analyser

In [6]:
authenticator = IAMAuthenticator('UzRvFLOEYA0HHDtWyEtPk4uScJQ1GD7_DzWF4zkypMBi')
tone_analyzer = ToneAnalyzerV3(
    version='2017-09-21',
    authenticator=authenticator
)

tone_analyzer.set_service_url('https://api.eu-gb.tone-analyzer.watson.cloud.ibm.com/instances/b7790d28-ef5f-4ad6-b8fe-e2d72753bead')
tone_analyzer.set_disable_ssl_verification(False)

In [7]:
tones = []
index = 0
for item in data['clean_tweet']:
    try:
        index = index + 1
        text = item
        tone_analysis = tone_analyzer.tone({'text': text}, content_type='application/json').get_result()
        tones.append(tone_analysis)
    except ApiException as ex:
        tones.append(None)
        print ("Method failed with status code " + str(ex.code) + ": " + ex.message)

In [8]:
tones_copy = tones.copy()
tones_copy = pd.DataFrame(tones_copy)

In [9]:
tones_copy.head()

,document_tone
0,"{'tones': [{'score': 0.622522, 'tone_id': 'ana..."
1,{'tones': []}
2,"{'tones': [{'score': 0.607756, 'tone_id': 'joy..."
3,{'tones': []}
4,{'tones': []}


In [10]:
data = data.reset_index()

In [11]:
data = data.drop(['index'], axis=1)

In [12]:
data = pd.concat([data,tones_copy],axis=1)
data.to_csv('covid1_500.csv',index=False)

# Formatting the document tones

In [13]:
data = pd.read_csv('covid1_500.csv')
data.head()

,User,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet,document_tone
0,bimaldas1965,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai,Pandemic is the result of loss of natural boun...,"{'tones': [{'score': 0.622522, 'tone_id': 'ana..."
1,freddzo,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai,they will ask me to visit the nearest branch a...,{'tones': []}
2,Andy_gawai,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai,No schedule for sleeping getting up having foo...,"{'tones': [{'score': 0.607756, 'tone_id': 'joy..."
3,AkshayMahapadi,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai,ji please declare d amendnent in ARP over to 3...,{'tones': []}
4,him_paliwal,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai,Have we got vaccine for Covid19 If not then wh...,{'tones': []}


In [14]:
data = data.iloc[:,1:26]

In [15]:
data['document_tones'] = data.document_tone.apply(lambda x: ast.literal_eval(x)['tones'])
data.drop('document_tone',axis = 1,inplace = True)

In [16]:
# split sentiments in corresponding columns
for i in range(0, len(data)):
    for j in range(0, len(data.loc[i, 'document_tones'])):
        dic = data.loc[i, 'document_tones'][j]
        source = dic['tone_id']
        data.loc[i, source] = 1
data.head()

,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet,document_tones,analytical,joy,sadness,tentative,confident,anger,fear
0,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai,Pandemic is the result of loss of natural boun...,"[{'score': 0.622522, 'tone_id': 'analytical', ...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai,they will ask me to visit the nearest branch a...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai,No schedule for sleeping getting up having foo...,"[{'score': 0.607756, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai,ji please declare d amendnent in ARP over to 3...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai,Have we got vaccine for Covid19 If not then wh...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:

data.to_csv('covid1_500_sentiment.csv',index=False)

# First 500 sentiments 
## Analytical ,joy, sadness, tentative, confident, anger, fear (Lockdown 1)

In [23]:
data

,Text,Date,Retweets,Favorites,HashTags,City,clean_tweet,document_tones,analytical,joy,sadness,tentative,confident,anger,fear
0,Pandemic is the result of loss of natural boun...,2020-04-13 22:56:36+00:00,0,1,#pandemic #covid19 #wildlife,Mumbai,Pandemic is the result of loss of natural boun...,"[{'score': 0.622522, 'tone_id': 'analytical', ...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,@AxisBank http://replied.Next they will ask me...,2020-04-13 22:27:59+00:00,0,0,#pathetic #bigticket #Help #COVID19,Mumbai,they will ask me to visit the nearest branch a...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,No schedule for sleeping/getting up/having foo...,2020-04-13 20:03:55+00:00,1,1,#IndiaFightsCorona #Covid19India #Covid_19 #CO...,Mumbai,No schedule for sleeping getting up having foo...,"[{'score': 0.607756, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
3,@PiyushGoyal ji please declare d amendnent in ...,2020-04-13 20:02:01+00:00,2,2,#Ganpati #lockdown #COVID19 #Konkan #Goa #Karn...,Mumbai,ji please declare d amendnent in ARP over to 3...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"Have we got vaccine for #Covid19? If not, then...",2020-04-13 18:49:53+00:00,2,2,#Covid19 #PleaseHelp #NeedtoSleep #CrackersKoS...,Mumbai,Have we got vaccine for Covid19 If not then wh...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,6 more #COVID19 positive cases ( 5 in Nainital...,2020-04-04 12:22:27+00:00,0,0,#COVID19,Mumbai,6 more COVID19 positive cases 5 in Nainital am...,"[{'score': 0.640469, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,NaN,1.0,NaN,NaN
496,#COVID2019 #COVID19Pandemic #Mumbai 48-year-Ol...,2020-04-04 12:15:34+00:00,0,2,#COVID2019 #COVID19Pandemic #Mumbai #Mukund #D...,Mumbai,COVID2019 COVID19Pandemic Mumbai 48 year Old M...,"[{'score': 0.58289, 'tone_id': 'analytical', '...",1.0,NaN,NaN,NaN,NaN,NaN,NaN
497,Currently we are seeing 2 epidemics... One of ...,2020-04-04 11:31:09+00:00,0,1,#COVID19 #STUPIDITY20,Mumbai,Currently we are seeing 2 epidemics One of cor...,[],NaN,NaN,NaN,NaN,NaN,NaN,NaN
498,"Stay home, stay safe! Brahminy Starling #color...",2020-04-04 11:12:14+00:00,1,2,#colors #beauty #nature #birds #birdwatching #...,Mumbai,Stay home stay safe Brahminy Starling colors a...,"[{'score': 0.730516, 'tone_id': 'joy', 'tone_n...",NaN,1.0,NaN,NaN,NaN,NaN,NaN
